In [1]:
# 한자 처리
import os

import hanja
import pandas as pd
from tqdm import tqdm

In [2]:
# 통화정책결정문 불러오기
flist = os.listdir("통화정책결정문")
flist = [f for f in flist if f[0] != "."]  # .으로 시작하는 파일 제외
# file = flist[0]

raw_datas = {}

for file in flist:  # 파일 하나씩 가져옴

    # 파일 불러옴
    with open(f"통화정책결정문/{file}") as f:
        lines = f.readlines()
        lines = [line.strip() for line in lines]  # 불필요한 줄바꿈, 공백 제외
        lines = [line for line in lines if line]  # 빈 라인 제외

    # 파일의 내용 딕셔너리에 정리
    data = {}

    cate = "[도입부]"
    data[cate] = []  # 빈 리스트 생성

    for line in lines:
        if line[0] == "[" and line[-1] == "]":  # 중괄호로 쌓인 내용의 경우
            cate = line  # 해당 라인을 카테고리명으로 사용
            
            if cate == "[통화정채방향]":  # 오타 수정
                cate = "[통화정책방향]"
            
            data[cate] = []  # 카테고리 리스트 생성

        else:  # 카테고리가 아닌 데이터 내용인 경우
            data[cate].append(line)  # 카테고리별로 한줄 추가
        
    # 처리한 내용 저장
    year = file[:4]
    month = file[4:6]
    day = file[6:8]
    
    new_name = f"{year}-{month}-{day}"
    raw_datas[new_name] = data


# csv로 저장해두기
merged_data = []
for k, v in raw_datas.items():
    row = {}
    row["날짜"] = k
    
    for cate, content in v.items():
        row[cate] = "\n".join(content)
    
    merged_data.append(row)

df = pd.DataFrame(merged_data)
# df.to_csv("통화정책방향_raw.csv", index=False)

merged_data[0]
# merged_data[-1]

{'날짜': '1999-01-07',
 '[도입부]': '1999.1.7일 금융통화위원회는 1999년 1월중 통화정책방향을 의결하였는데, 그 주요내용은 붙임과 같음',
 '[통화정책방향]': '국내경제는 지난 11월중 수출증대에 힘입어 산업생산이 소폭 증가로 반전되고 소비 및 투자 등 내수도 감소세가 둔화되는 등 경기가 다소 호전되는 움직임을 나타내었음\n외환시장에서는 경상수지 흑자가 지속되고 외국인투자자금이 꾸준히 유입됨으로써 외환공급 우위기조가 강화되어 원화환율이 큰 폭으로 하락하고 가용외환보유액이 98년 12월말 현재 500억달러에 육박하였음\n금융시장에서는 물가 및 환율이 안정된 가운데 공개시장조작금리 인하와 향후 경기부양을 위한 확장적 경제정책 운용기대로 장기시장금리가 큰 폭으로 하락하고 은행대출금리도 꾸준히 낮아지면서\n상업어음할인 등 중소기업대출이 다소 회복되고 CP 및 회사채 발행에 있어 발행대상기업이 확대되고 신용등급별 금리격차가 꾸준히 축소되는 등 신용경색 현상이 완화되는 모습을 보이고 있음\n한편 강한 금리하락 기대와 낙관적 경제전망 등의 영향으로 금융저축 자금의 일부가 주식시장으로 이동하면서 주가가 급등하는 등 자산가격이 상승하는 움직임을 보였음\n앞으로도 외환공급우위로 환율절상 압력을 받고 이에 따라 금리하락 기대가 이어지는 한편 경기회복과정에서 풍부한 유동성과 저금리를 바탕으로 자산가격이 불안한 움직임을 보일 가능성이 있음\n이러한 점을 감안할 때 적절한 외환수급조절을 통해 환율절상 압력이 과도히 커지지 않도록 하여 통화정책이 국내경제여건을 고려하여 탄력적으로 이루어질 수 있도록 노력하는 한편\n금융자금 이동 및 자산시장 동향을 주의깊게 관찰하면서 그간의 금융완화정책이 실물경제에 파급되는 효과를 면밀히 분석할 필요가 있음\n이에 따라 1월중 공개시장조작은 외환시장 및 금융시장 여건을 보아가며 콜금리를 약간 하향조정된 수준에서 안정되도록 운영하는 한편 본원통화를 은행의 대출자금수요에 맞추어 공급하는 방향으로 운영함'}

In [3]:
len(merged_data)

268

In [4]:
# 통화정책방향_processed에 저장함, 한자가 있을 경우 '한자처리' 폴더에 저장해놓고 한번 더 처리해서 저장
for data in tqdm(merged_data):
    filename = data["날짜"]
    script = data["[통화정책방향]"]
    
    # 한자 포함 여부 체크
    translate = False
    for alp in script:
        if hanja.is_hanja(alp):
            translate = True
            break
    
    if translate:  # 한자 존재 시 혼용변환으로 우선 처리 (완벽하진 않음)
        script = hanja.translate(script, 'combination-text')
        filename = "한자처리/" + filename  # 한자 처리해야 하는 파일들은 여기에 저장 >> 수동으로 처리한 후 옮기기
        
    # 저장
    with open(f"통화정책방향_processed/{filename}.txt", "w") as f:
        f.write(script)

100%|██████████| 268/268 [00:02<00:00, 101.13it/s]
